### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [2]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
# query_url = (
#     f"{base_url}api-key={NASA_API_KEY}&begin_date={startDate}&end_date={endDate}"
#     + f'&fq={filter_query}&sort={sort}&fl={field_list}'
# )
# https://api.nasa.gov/DONKI/CME?startDate=yyyy-MM-dd&endDate=yyyy-MM-dd&api_key=DEMO_KEY
cme_url = (
    f"{base_url}{CME}?startDate={startDate}&end_date={endDate}&api_key={NASA_API_KEY}"
)

In [3]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
response = requests.get(cme_url)
response

<Response [200]>

In [4]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = response.json()


In [5]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [58]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
#cmd_df = cme_df[["activityID", "startTime", "linkedEvents"]]
cme_df = cme_df.loc[:,["activityID", "startTime", "linkedEvents"]]
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [68]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme = cme_df.dropna()
cme.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [74]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme.index:
    # Get the corresponding value from row i in 'activityID'
    activityID = cme['activityID'][i]
    # Get the corresponding value from row i in 'startTime'     
    startTime = cme['startTime'][i]
    # Get the list of dictionaries from row i in 'linkedEvents'
    linkedEvents =cme['linkedEvents'][i]
    
    # Iterate over each dictionary in the list
    for item in linkedEvents:
    # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({'activityID': activityID, 'startTime': startTime, 'linkedEvents': linkedEvents})



# Create a new DataFrame from the expanded rows
cme_expanded_df = pd.DataFrame(expanded_rows)
cme_expanded_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...


In [92]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
        # Log the error or print it for debugging
def extract_activityID_from_dict(input_dict):
    try:
        # print(input_dict)
        # print(input_dict[0].values())
        #input_dict[0].values()
        woot = input_dict[0].values()
        return woot
    except (ValueError, TypeError) as e:
        print(f"Error with dictionary: {input_dict}")
        return 

extract_activityID_from_dict(cme.loc[0,'linkedEvents'])


dict_values(['2013-05-04T04:52:00-IPS-001'])

In [110]:
extract_activityID_from_dict(cme.loc[0,'linkedEvents'])

dict_values(['2013-05-04T04:52:00-IPS-001'])

In [104]:
cme.loc[0,'linkedEvents']

[{'activityID': '2013-05-04T04:52:00-IPS-001'}]

In [105]:
cme["linkedEvents"][0]

[{'activityID': '2013-05-04T04:52:00-IPS-001'}]

In [109]:
extract_activityID_from_dict(cme['linkedEvents'][0])

dict_values(['2013-05-04T04:52:00-IPS-001'])

In [94]:
cme_expanded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2139 entries, 0 to 2138
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   activityID    2139 non-null   object
 1   startTime     2139 non-null   object
 2   linkedEvents  2139 non-null   object
dtypes: object(3)
memory usage: 50.3+ KB


In [ ]:
# df = df.assign(Percentage = lambda x: (x['Total_Marks'] /500 * 100))
# #df = pd.DataFrame({
#     'A': [1, 2, 3],
#     'B': [4, 5, 6]
# })

# # Apply a lambda function to each element
# df['A_plus_B'] = df.apply(lambda row: row['A'] + row['B'], axis=1)
# print(df)
cme_expanded_df["hold"] = df['A'].apply(lambda x: x ** 2)
extract_activityID_from_dict(cme.loc[0,'linkedEvents'])
df_expanded.loc[:, 'GST_ActivityID'] = df_expanded['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x[0]) if isinstance (x, list) and len(x) > 0 else None)
cme_expanded_df["hold"] = cme_expanded_df.apply(lambda x: cme.loc[x,'linkedEvents'])
 extract_activityID_from_dict(input_dict)
df['col_3'] = df.apply(lambda x: get_sublist(x.col_1, x.col_2), axis=1)
cme_expanded_df["hold"] =  cme.apply(lambda x: extract_activityID_from_dict(cme['linkedEvents']), axis =1)

In [120]:
cme.apply(lambda row: extract_activityID_from_dict(row['linkedEvents']), axis =1)

0       (2013-05-04T04:52:00-IPS-001)
4       (2013-05-07T04:37:00-IPS-001)
7       (2013-05-12T23:30:00-IPS-001)
10      (2013-05-13T01:53:00-FLR-001)
13      (2013-05-13T15:40:00-FLR-001)
                    ...              
6404    (2024-11-29T00:40:00-IPS-001)
6405    (2024-11-29T00:40:00-IPS-001)
6409    (2024-11-25T20:24:00-FLR-001)
6412    (2024-11-27T02:21:00-FLR-001)
6414    (2024-12-02T20:07:00-IPS-001)
Length: 1231, dtype: object

In [103]:
cme_expanded_df["hold"] =  cme.apply(lambda row: extract_activityID_from_dict(row['linkedEvents']), axis =1)

AttributeError: 'list' object has no attribute 'values'

In [95]:
cme.loc[0,'linkedEvents']

[{'activityID': '2013-05-04T04:52:00-IPS-001'}]

In [ ]:
df = df.apply(lambda x: np.square(x) if x.name in [
              'a', 'e', 'g'] else x, axis=1)

In [ ]:
df = df.apply(lambda x: np.square(x) if x.name in [
              'a', 'e', 'g'] else x, axis=1)

In [83]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:


In [11]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:


In [12]:
# print out the datatype of each column in this DataFrame:


In [13]:
# Convert the 'GST_ActivityID' column to string format 

# Convert startTime to datetime format  

# Rename startTime to startTime_CME and activityID to cmeID

# Drop linkedEvents

# Verify that all steps were executed correctly


In [14]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  


### GST Data

In [29]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = (
    f"{base_url}GST?startDate={startDate}&end_date={endDate}&api_key={NASA_API_KEY}"
)

In [30]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)
gst_response

<Response [200]>

In [17]:
# Convert the response variable to json and store it as a variable named gst_json

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data


In [18]:
# Convert gst_json to a Pandas DataFrame  

# Keep only the columns: activityID, startTime, linkedEvents


In [19]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME


In [20]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.


In [21]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:


In [22]:
# Convert the 'CME_ActivityID' column to string format 

# Convert the 'gstID' column to string format 

# Convert startTime to datetime format  

# Rename startTime to startTime_GST 

# Drop linkedEvents

# Verify that all steps were executed correctly


In [23]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  


### Merge both datatsets

In [24]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.


In [25]:
# Verify that the new DataFrame has the same number of rows as cme and gst


### Computing the time it takes for a CME to cause a GST

In [26]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.


In [27]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 


### Exporting data in csv format

In [28]:
# Export data to CSV without the index
